## Import Libraries

In [15]:
import re
import json
from collections import defaultdict, Counter
from datasets import load_dataset
import pickle

In [18]:
class SimpleBPE:
    def __init__(self, vocab_size=10000):
        self.vocab_size = vocab_size
        self.word_freqs = {}
        self.vocab = {}
        self.merges = []
        
    def get_stats(self, word_freqs):
        pairs = defaultdict(int)
        for word, freq in word_freqs.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[(symbols[i], symbols[i + 1])] += freq
        return pairs
    
    def merge_vocab(self, pair, word_freqs):
        new_word_freqs = {}
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
        
        for word in word_freqs:
            new_word = p.sub(''.join(pair), word)
            new_word_freqs[new_word] = word_freqs[word]
        return new_word_freqs
    
    def get_word_tokens(self, text):
        # Simple tokenization - split on whitespace and punctuation
        words = re.findall(r'\b\w+\b', text.lower())
        word_freqs = Counter(words)
        
        # Add end-of-word marker and split into characters
        processed_words = {}
        for word, freq in word_freqs.items():
            processed_words[' '.join(word) + ' </w>'] = freq
        
        return processed_words
    
    def train(self, texts):
        print("Processing texts and building word frequencies...")
        
        # Collect all words and their frequencies
        all_word_freqs = defaultdict(int)
        for text in texts:
            word_freqs = self.get_word_tokens(text)
            for word, freq in word_freqs.items():
                all_word_freqs[word] += freq
        
        self.word_freqs = dict(all_word_freqs)
        print(f"Found {len(self.word_freqs)} unique words")
        
        # Get initial vocabulary (all characters)
        vocab = set()
        for word in self.word_freqs.keys():
            vocab.update(word.split())
        
        # Create initial vocab with indices
        self.vocab = {token: i for i, token in enumerate(sorted(vocab))}
        print(f"Initial vocabulary size: {len(self.vocab)}")
        
        # Perform BPE merges
        num_merges = self.vocab_size - len(self.vocab)
        print(f"Performing {num_merges} merges...")
        
        for i in range(num_merges):
            pairs = self.get_stats(self.word_freqs)
            if not pairs:
                break
                
            best_pair = max(pairs, key=pairs.get)
            self.word_freqs = self.merge_vocab(best_pair, self.word_freqs)
            self.merges.append(best_pair)
            
            # Add new token to vocabulary
            new_token = ''.join(best_pair)
            self.vocab[new_token] = len(self.vocab)
            
            if (i + 1) % 1000 == 0:
                print(f"Completed {i + 1} merges, current vocab size: {len(self.vocab)}")
        
        print(f"Training complete! Final vocabulary size: {len(self.vocab)}")
    
    def encode_word(self, word):
        word = ' '.join(word) + ' </w>'
        
        # Apply merges in order
        for pair in self.merges:
            bigram = re.escape(' '.join(pair))
            p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
            word = p.sub(''.join(pair), word)
        
        return word.split()
    
    def encode(self, text):
        words = re.findall(r'\b\w+\b', text.lower())
        tokens = []
        
        for word in words:
            word_tokens = self.encode_word(word)
            tokens.extend(word_tokens)
        
        return tokens
    
    def get_vocab_tokens(self, min_length=3):
        # Create reverse vocab for easy lookup
        reverse_vocab = {v: k for k, v in self.vocab.items()}
        
        # Get tokens with their characteristics
        tokens_info = []
        for idx, token in reverse_vocab.items():
            if len(token) >= min_length and not token.endswith('</w>'):
                tokens_info.append({
                    'token': token,
                    'length': len(token),
                    'is_word_end': token.endswith('</w>'),
                    'index': idx
                })
        
        # Sort by length (descending) then alphabetically
        tokens_info.sort(key=lambda x: (-x['length'], x['token']))
        return tokens_info


In [19]:
def analyze_vocabulary(bpe_tokenizer, sample_size=50):
    print("\n=== VOCABULARY ANALYSIS ===")
    
    vocab_tokens = bpe_tokenizer.get_vocab_tokens()
    
    print(f"Total vocabulary size: {len(bpe_tokenizer.vocab)}")
    print(f"Tokens with length >= 3: {len(vocab_tokens)}")
    
    print(f"\nTop {sample_size} longest tokens:")
    for i, token_info in enumerate(vocab_tokens[:sample_size]):
        print(f"{i+1:2d}. '{token_info['token']}' (length: {token_info['length']})")
    
    # Analyze token length distribution
    length_dist = defaultdict(int)
    for token in bpe_tokenizer.vocab.keys():
        length_dist[len(token)] += 1
    
    print(f"\nToken length distribution:")
    for length in sorted(length_dist.keys()):
        print(f"Length {length}: {length_dist[length]} tokens")

In [20]:
def test_on_ml_papers(bpe_tokenizer, num_samples=10):
    print("\n=== TESTING ON ML PAPERS ===")
    
    # Try to load ML papers dataset
    try:
        print("Attempting to load ML papers dataset...")
        dataset = load_dataset("CShorten/ML-ArXiv-Papers", split="train", streaming=True)
        papers = []
        
        print("Loading ML papers sample...")
        for i, paper in enumerate(dataset):
            if i >= num_samples:
                break
            # Combine title and abstract for analysis
            title = paper.get('title', '')
            abstract = paper.get('abstract', '')
            if title and abstract:
                text = title + ' ' + abstract
                papers.append(text)
        
        print(f"Loaded {len(papers)} papers")
        
    except Exception as e:
        print(f"ML papers dataset not available: {e}")
        print("Using sample ML paper texts...")
        
        # Sample ML paper abstracts for testing
        papers = [
            """Attention Is All You Need: We propose a new simple network architecture, the Transformer, 
            based solely on attention mechanisms, dispensing with recurrence and convolutions entirely. 
            Experiments on two machine translation tasks show these models to be superior in quality 
            while being more parallelizable and requiring significantly less time to train.""",
            
            """BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding: We introduce 
            a new language representation model called BERT, which stands for Bidirectional Encoder 
            Representations from Transformers. BERT is designed to pre-train deep bidirectional 
            representations from unlabeled text by jointly conditioning on both left and right context.""",
            
            """Deep Residual Learning for Image Recognition: We present a residual learning framework 
            to ease the training of networks that are substantially deeper than those used previously. 
            We explicitly reformulate the layers as learning residual functions with reference to the 
            layer inputs, instead of learning unreferenced functions.""",
            
            """Generative Adversarial Networks: We propose a new framework for estimating generative 
            models via an adversarial process, in which we simultaneously train two models: a generative 
            model G that captures the data distribution, and a discriminative model D that estimates 
            the probability that a sample came from the training data.""",
            
            """Adam: A Method for Stochastic Optimization: We introduce Adam, an algorithm for first-order 
            gradient-based optimization of stochastic objective functions, based on adaptive estimates 
            of lower-order moments. The method is straightforward to implement, is computationally 
            efficient, and has little memory requirements."""
        ]
    
    # Test tokenization on technical terms
    technical_terms = [
        'neural network', 'machine learning', 'deep learning', 'transformer',
        'attention mechanism', 'backpropagation', 'gradient descent', 
        'convolutional', 'recurrent', 'reinforcement learning',
        'natural language processing', 'computer vision', 'optimization',
        'generative adversarial', 'residual learning', 'bidirectional'
    ]
    
    print("\nTechnical term tokenization:")
    for term in technical_terms:
        tokens = bpe_tokenizer.encode(term)
        print(f"'{term}' -> {tokens}")
    
    # Analyze a sample paper
    if papers:
        print(f"\nSample paper tokenization:")
        sample_text = papers[0][:300] + "..."  # First 300 chars
        tokens = bpe_tokenizer.encode(sample_text)
        print(f"Text: {sample_text}")
        print(f"Tokens ({len(tokens)}): {tokens[:25]}...")  # First 25 tokens


In [21]:
def main():
    print("=== SIMPLE BPE TOKENIZER IMPLEMENTATION ===")
    
    # Initialize BPE tokenizer
    bpe = SimpleBPE(vocab_size=10000)
    
    # Try multiple dataset options
    print("Loading training dataset...")
    texts = []
    
    # Option 1: Try OpenWebText
    try:
        print("Attempting to load OpenWebText dataset...")
        dataset = load_dataset("openwebtext", split="train", streaming=True)
        
        print("Collecting training texts from OpenWebText...")
        for i, example in enumerate(dataset):
            if i >= 500:  # Use first 500 documents
                break
            if len(example['text']) > 100:  # Skip very short texts
                texts.append(example['text'])
            if (i + 1) % 50 == 0:
                print(f"Loaded {i + 1} documents...")
        
        print(f"Successfully loaded {len(texts)} texts from OpenWebText")
        
    except Exception as e:
        print(f"OpenWebText not available: {e}")
        
        # Option 2: Try BookCorpus
        try:
            print("Attempting to load BookCorpus dataset...")
            dataset = load_dataset("bookcorpus", split="train", streaming=True)
            
            print("Collecting training texts from BookCorpus...")
            for i, example in enumerate(dataset):
                if i >= 500:
                    break
                if len(example['text']) > 100:
                    texts.append(example['text'])
                if (i + 1) % 50 == 0:
                    print(f"Loaded {i + 1} books...")
            
            print(f"Successfully loaded {len(texts)} texts from BookCorpus")
            
        except Exception as e:
            print(f"BookCorpus not available: {e}")
            
            # Option 3: Try C4 (Common Crawl)
            try:
                print("Attempting to load C4 dataset...")
                dataset = load_dataset("c4", "en", split="train", streaming=True)
                
                print("Collecting training texts from C4...")
                for i, example in enumerate(dataset):
                    if i >= 300:
                        break
                    if len(example['text']) > 200:
                        texts.append(example['text'])
                    if (i + 1) % 50 == 0:
                        print(f"Loaded {i + 1} documents...")
                
                print(f"Successfully loaded {len(texts)} texts from C4")
                
            except Exception as e:
                print(f"C4 not available: {e}")
                
                # Option 4: Try Wikitext
                try:
                    print("Attempting to load Wikitext dataset...")
                    dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="train")
                    
                    print("Processing Wikitext data...")
                    # Wikitext comes as one big text, split it into chunks
                    full_text = dataset['text']
                    text_chunks = []
                    current_chunk = ""
                    
                    for line in full_text:
                        if line.strip():
                            current_chunk += line + " "
                            if len(current_chunk) > 1000:  # Create chunks of ~1000 chars
                                text_chunks.append(current_chunk.strip())
                                current_chunk = ""
                    
                    texts = text_chunks[:1000]  # Take first 1000 chunks
                    print(f"Successfully loaded {len(texts)} text chunks from Wikitext")
                    
                except Exception as e:
                    print(f"Wikitext not available: {e}")
                    print("Using comprehensive sample texts for demonstration...")
                    
                    # Extensive fallback sample texts
                    texts = [
                        """The development of artificial intelligence has been one of the most significant technological 
                        advances of the 21st century. Machine learning algorithms can now process vast amounts of data 
                        and identify complex patterns that would be impossible for humans to detect manually.""",
                        
                        """Natural language processing involves the computational analysis of human language. Modern NLP 
                        systems use transformer architectures and attention mechanisms to understand context and generate 
                        coherent responses to user queries.""",
                        
                        """Computer vision systems have revolutionized image recognition and object detection. Convolutional 
                        neural networks can now achieve superhuman performance on many visual recognition tasks.""",
                        
                        """Deep learning models require massive computational resources for training. Graphics processing 
                        units and specialized tensor processing units have become essential hardware for machine learning 
                        research and deployment.""",
                        
                        """Reinforcement learning algorithms learn through interaction with their environment. These systems 
                        can master complex games and control systems by optimizing reward signals over time.""",
                        
                        """The transformer architecture introduced the concept of self-attention, allowing models to focus 
                        on different parts of the input sequence when making predictions. This breakthrough enabled the 
                        development of large language models.""",
                        
                        """Gradient descent optimization algorithms are fundamental to training neural networks. Variants 
                        like Adam and RMSprop help models converge more efficiently by adapting learning rates during 
                        training.""",
                        
                        """Regularization techniques like dropout and batch normalization help prevent overfitting in 
                        deep neural networks. These methods improve generalization performance on unseen data.""",
                        
                        """Transfer learning allows models trained on large datasets to be fine-tuned for specific tasks 
                        with limited data. This approach has democratized access to powerful machine learning capabilities.""",
                        
                        """Ensemble methods combine multiple models to improve prediction accuracy and robustness. Random 
                        forests and gradient boosting are popular ensemble techniques in machine learning."""
                    ] * 100  # Repeat for more diverse training data
    
    # Train the BPE tokenizer
    bpe.train(texts)
    
    # Analyze the vocabulary
    analyze_vocabulary(bpe)
    
    # Test on ML papers
    test_on_ml_papers(bpe)
    
    # Save the tokenizer
    print("\nSaving tokenizer...")
    with open('bpe_tokenizer.pkl', 'wb') as f:
        pickle.dump(bpe, f)
    
    print("BPE tokenizer saved as 'bpe_tokenizer.pkl'")
    
    return bpe

In [22]:
if __name__ == "__main__":
    tokenizer = main()

=== SIMPLE BPE TOKENIZER IMPLEMENTATION ===
Loading training dataset...
Attempting to load OpenWebText dataset...


README.md:   0%|          | 0.00/7.35k [00:00<?, ?B/s]

C:\Users\zyad3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zyad3\.cache\huggingface\hub\datasets--openwebtext. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


openwebtext.py:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

OpenWebText not available: The repository for openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/openwebtext.
Please pass the argument `trust_remote_code=True` to allow custom code to be run.
Attempting to load BookCorpus dataset...


...

c4.py:   0%|          | 0.00/3.46k [00:00<?, ?B/s]

C4 not available: The repository for c4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/c4.
Please pass the argument `trust_remote_code=True` to allow custom code to be run.
Attempting to load Wikitext dataset...


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

C:\Users\zyad3\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zyad3\.cache\huggingface\hub\datasets--wikitext. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Wikitext not available: (ReadTimeoutError("HTTPSConnectionPool(host='cdn-lfs.hf.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 6afc159a-a7b1-432f-9ec8-cea3a722e83b)')
Using comprehensive sample texts for demonstration...
Processing texts and building word frequencies...
Found 176 unique words
Initial vocabulary size: 29
Performing 9971 merges...
Training complete! Final vocabulary size: 635

=== VOCABULARY ANALYSIS ===
Total vocabulary size: 635
Tokens with length >= 3: 319

Top 50 longest tokens:
 1. 'architecture' (length: 12)
 2. 'architectur' (length: 11)
 3. 'technologic' (length: 11)
 4. 'architectu' (length: 10)
 5. 'capabiliti' (length: 10)
 6. 'democratiz' (length: 10)
 7. 'intelligen' (length: 10)
 8. 'prediction' (length: 10)
 9. 'revolution' (length: 10)
10. 'significan' (length: 10)
11. 'architect' (length: 9)
12. 'democrati' (length: 9)
13. 'efficient' (length: 9)
14. 'fundament' (length: 9)
15. 'introduce' (length: 9)
16. 'performan' (length: 9)
17. '